In [5]:
import pandas as pd

# 1. Load the dataset
df = pd.read_csv('products_data.csv')

# 2. Replace '?' with '₱' in the Product_Price column
# We use regex=False because '?' is normally a special character in Python searches
df['Product_Price'] = df['Product_Price'].str.replace('?', '₱', regex=False)

# 3. Save the updated DataFrame back to a CSV file
# IMPORTANT: 'utf-8-sig' is the key to making the Peso sign show correctly in Excel
df.to_csv('products_data.csv', index=False, encoding='utf-8-sig')

In [9]:
import numpy as np

df = pd.read_csv('transactions_data.csv')

# Drop the leftover index column
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [11]:
# Convert to a single datetime format
# 'format=mixed' tells pandas to handle multiple different string formats at once
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], errors='coerce', format='mixed')

In [13]:
# 1. Fill missing Total_Cost
mask_cost = df['Total_Cost'].isnull() & df['Quantity'].notnull() & df['Product_Price'].notnull()
df.loc[mask_cost, 'Total_Cost'] = df.loc[mask_cost, 'Quantity'] * df.loc[mask_cost, 'Product_Price']

# 2. Fill missing Quantity
mask_qty = df['Quantity'].isnull() & df['Total_Cost'].notnull() & df['Product_Price'].notnull()
df.loc[mask_qty, 'Quantity'] = (df.loc[mask_qty, 'Total_Cost'] / df.loc[mask_qty, 'Product_Price']).round()

In [15]:
# Fill missing IDs with 0 and convert to integer
df['Transaction_ID'] = df['Transaction_ID'].fillna(0).astype(int)
df['Company_ID'] = df['Company_ID'].fillna(0).astype(int)
df['Product_ID'] = df['Product_ID'].fillna(0).astype(int)

In [19]:
# Drop rows where more than half the data is missing
df = df.dropna(thresh=5) 

# Sort by Date
df = df.sort_values(by='Transaction_Date')

# Save as a cleaned file
df.to_csv('transactions_data.csv', index=False)